In [33]:
import pandas as pd
import numpy as np
import os, math, sys
import pickle

In [34]:
# defaults:
def digitLineByLine(data, d):
    info = []
    for index, row in data.iterrows():
        test = []
        for col in data:
            item = row[col]
            test.append(d[item])
        info.append(test)
    return info

def lineByLine(data, d):
    info = []
    for index, row in data.iterrows():
        test = []
        for col in data:
            item = row[col]
            if type(d[col][item]) is list:
                test.extend(d[col][item])
            else:
                test.append(d[col][item])
        info.append(test)
    return info

def oneHot(data, col, d):
    itemList = pd.unique(data[[col]].values.ravel('K'))
    count = 0
    for currItem in itemList:
        count = count + 1
        array = np.identity(d[col]['count'])
#         print(currItem + ": "+ str(array[count-1:count,:]))
        d[col][currItem]=array[count-1:count,:].tolist()[0]
        
    return d

def normalize(data):
    itemList = pd.unique(data.values.ravel('K'))
    x = {}
    for currItem in itemList:
        new = (currItem - min(itemList))/(max(itemList)-min(itemList))
        x[currItem] = new
    return x

def isDigits(data):
    dc = 0
    for item in data:
        if type(item) is int:
            dc = dc +1
    if dc == len(data):
        return True
    else:
        return False

def detectTypes(data):
    d = {}
    
    if isDigits(pd.unique(data.values.ravel('K')).tolist()):
        d = normalize(data)
    else: 
        for col in data.columns.values.tolist():
    #         print(col)
            d[col] = {"count":len(data[col].unique().tolist())}
            # check types
            for item in pd.unique(data[[col]].values.ravel('K')):
    #             print("\t"+item)
                if d[col]["count"] == 2:
                    # binary attribute
                    if 0 in d[col].values():
                        d[col][item] = 1.0
                    else:
                        d[col][item] = 0
                else:
                    d = oneHot(data, col, d)
    
    return d

def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def saveFrame(df, name):
    df.to_csv(name+".csv", index=False, sep=",", header=True)
    save_it_all(df, name+".pkl")
    
def read(filename):
    with open(filename) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content] 
    return content

def write(my_list, name):
    with open(name+'.csv', 'w') as f:
        for item in my_list:
            f.write("%s\n" % item)
            
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def onehotTargets(targets):
    #for target in targets.tolist():
    targetSet = list(set(targets.tolist()))
    dicT = {}
    count = 0
    for currItem in targetSet:
        count = count + 1
        array = np.identity(len(targetSet))
    #         print(currItem + ": "+ str(array[count-1:count,:]))
        dicT[currItem]=array[count-1:count,:].tolist()[0]

    return dicT

When encoding binary and categorical data, there are four cases you must deal with: independent (x) binary data, dependent (y) binary data, independent (x) categorical data and dependent (y) categorical data. 



Wind,2,Strong,Weak
Water,3,Warm,Moderate,Cold
Air,2,Warm,Cool
Forecast,3,Sunny,Cloudy,Rainy


In [35]:
content = read("fishingNN.data")
header = "wind,water,air,forcast,class"
header = [header] + content
header
write(header, "FishEdit")

In [36]:
data = pd.read_csv('FishEdit.csv', sep=",")
x = detectTypes(data)
x["class"]["Yes"] = 1 
x["class"]["No"] = 0 
save_it_all(x, "./FishCleanDict.pkl")
new = lineByLine(data, x)
df = pd.DataFrame(new, columns=range(0,len(new[0])))

In [37]:
df.to_csv("normalizeFish.csv", sep=',', index=None, header=None)   
test = pd.read_csv("normalizeFish.csv", sep=',', header=None)
test

,0,1,2,3,4,5,6,7,8
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0
5,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0
6,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
7,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
8,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1
9,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0


Read in the digit set


In [38]:
# digitDataWithClass = pd.read_csv("digits-training.data", sep=" ", header=None)
# digitData = digitDataWithClass.loc[:,:64].copy()
digitDataTotal = pd.read_csv("digits-training.data", sep=" ", header=None)
x = detectTypes(digitDataTotal)

new = digitLineByLine(digitDataTotal, x)
df = pd.DataFrame(new, columns=range(0,len(new[0])))

In [39]:
df
df.to_csv("normDigit.csv", sep=',', index=None, header=None)   
test = pd.read_csv("normDigit.csv", sep=',', header=None)


## process the test sets:


In [40]:
testFish = pd.read_csv('fishTest.csv', sep=",")
testFish

cleanDict = load_objects("./FishCleanDict.pkl")
cleanDict

cleanTest = lineByLine(testFish, cleanDict)
cleanTest

dfx = pd.DataFrame(cleanTest, columns=range(0,len(cleanTest[0])))

dfx.to_csv("normalizeFishTest.csv", sep=',', index=None, header=None)   
test = pd.read_csv("normalizeFishTest.csv", sep=',', header=None)
test

,0,1,2,3,4,5,6,7
0,0,0.0,0.0,1.0,0,1.0,0.0,0.0


# One hot encode digit targets

In [42]:
digitDataTotal = pd.read_csv("digits-training.data", sep=" ", header=None)

targets = digitDataTotal[64]
features = digitDataTotal.iloc[:,0:64]


x = detectTypes(features)
save_it_all(x, "./digitCleanDict.pkl")

f = onehotTargets(targets)
save_it_all(f, "./digitTargetCleanDict.pkl")

new = digitLineByLine(features, x)

df = pd.DataFrame(new, columns=range(0,len(new[0])))
df[64] = targets.tolist()

df.to_csv("normDigit.csv", sep=',', index=None, header=None)   
test = pd.read_csv("normDigit.csv", sep=',', header=None)
test

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0.0,0.0625,0.3750,0.9375,0.7500,0.0625,0.0000,0.0000,0.0,0.4375,...,0.0000,0.0,0.0000,0.3750,0.8750,0.4375,0.0625,0.0000,0.000,0
1,0.0,0.0000,0.6250,1.0000,0.3750,0.0000,0.0000,0.0000,0.0,0.4375,...,0.0000,0.0,0.0000,0.6250,1.0000,0.9375,0.1875,0.0000,0.000,0
2,0.0,0.0000,0.5000,0.9375,1.0000,0.8125,0.0000,0.0000,0.0,0.0625,...,0.0000,0.0,0.0000,0.5625,0.8750,0.0000,0.0000,0.0000,0.000,7
3,0.0,0.0000,0.0000,0.1875,0.6875,1.0000,0.0000,0.0000,0.0,0.0000,...,0.0000,0.0,0.0000,0.0000,0.0625,0.9375,0.1250,0.0000,0.000,4
4,0.0,0.0000,0.3125,0.8750,0.2500,0.0000,0.0000,0.0000,0.0,0.0000,...,0.0000,0.0,0.0000,0.2500,0.7500,0.8750,0.4375,0.0000,0.000,6
5,0.0,0.0000,0.6875,1.0000,0.6250,0.0625,0.0000,0.0000,0.0,0.2500,...,0.1875,0.0,0.0000,0.6250,1.0000,1.0000,1.0000,1.0000,0.375,2
6,0.0,0.0000,0.0625,0.6875,0.8125,0.6875,0.4375,0.0000,0.0,0.0000,...,0.0000,0.0,0.0000,0.0625,0.8125,0.3125,0.0000,0.0000,0.000,5
7,0.0,0.0000,0.5000,0.6250,0.5000,0.4375,0.1250,0.0000,0.0,0.0625,...,0.0000,0.0,0.0000,0.2500,0.8125,0.5000,0.0000,0.0000,0.000,5
8,0.0,0.0000,0.9375,0.1250,0.8750,0.8125,0.1250,0.0000,0.0,0.0000,...,0.0000,0.0,0.0000,0.6250,0.7500,0.3125,0.0000,0.0000,0.000,0
9,0.0,0.0000,0.1875,0.8125,0.8125,0.1250,0.0000,0.0000,0.0,0.3750,...,0.0000,0.0,0.0000,0.1875,0.9375,0.6875,0.3750,0.0000,0.000,8
